
(tutorials-hubbard-selfconsistent)=

# Computing Hubbard parameters self-consistently

In this tutorial you will learn how to compute iteratively the Hubbard parameters through the {py:class}`~aiida_hubbard.workflows.hubbard.SelfConsistentHubbardWorkChain`.

In [ ]:
from local_module import load_temp_profile
from aiida_quantumespresso.data.hubbard_structure import HubbardStructureData

# If you download this file, you can run it with your own profile.
# Put these lines instead:
# from aiida import load_profile
# load_profile()
data = load_temp_profile(
    name="hubbard-selfconsistent-tutorial",
    add_computer=True,
    add_pw_code=True,
    add_hp_code=True,
    add_sssp=True,
)

# We initialize only the U, so that `hp.x` will understand it
# needs to compute only the onsite parameters.
a, b, c, d = 1.40803, 0.81293, 4.68453, 1.62585
cell = [[a, -b, c], [0.0, d, c], [-a, -b, c]]
sites = [
    ['Co', 'Co', (0, 0, 0)],
    ['O',   'O', (0, 0, 3.6608)], 
    ['O',   'O', (0, 0, 10.392)], 
    ['Li', 'Li', (0, 0, 7.0268)],
]

hubbard_structure = HubbardStructureData(cell=cell, sites=sites)
hubbard_structure.initialize_onsites_hubbard("Co", "3d")
hubbard_structure.store()

## The cycle

To have a full ab-initio calculation of Hubbard parameters, an iterative procedure should be employed. This forsees the following steps, to do in a cyclic way till the parameters don't differ from the previous ones by a certain threshold, i.e. ___self-consistently___.

The steps to do in order are:
1. Perform a volume relaxation of the structure, starting from a zero value of Hubbard parameters (i.e. if it was a 'non-Hubbard' calculation).
2. Perform the ground-state calculation (SCF) of the relaxed structure.
3. Perform the linear response calculation to predict the new Hubbard values.
4. If _all_ U (and V) are within the desired threshold, stop, otherwise restart with the new values from (1).

::: {admonition} Note for SCF (step 2)
:class: note

Tipically, as these are electronic responses, the gound-state SCF can be performed _with looser energy cutoffs and k poit density_, and still retain the same accuracy on the prediction of Hubbard parameters. 

```{important}
Before any production run, you should make sure to have converged such parameters.
```
:::

::: {admonition} Note for thresholds
:class: note

Threshold for U and V may depend on the final goal, or property, of your research. From our experience, good values are of the order of 0.1 eV for the onsite parameters (U) and 0.01 eV for the intersites (V).
:::

### Automating the cycle

As we already learnt from the previous tutorials ([1](./1_computing_hubbard.ipynb),[2](./2_parallel_hubbard.ipynb)), we can simply fill the builder of the work chain using the `get_builder_from_protocol` to get to know what the workflow is doing, and how this can help  and speed up our research.

:::{warning}
In this tutorial we will compute only the U on Co, and not the V for Co-O. This is to speed up the simulation, which on only a handful of cores would take tens of minutes, if not more.

This workflow may take 5 minutes (or more) to complete depending on your local resources.
:::

In [ ]:
from aiida.engine import run_get_node
from aiida_hubbard.workflows.hubbard import SelfConsistentHubbardWorkChain

builder = SelfConsistentHubbardWorkChain.get_builder_from_protocol(
    pw_code=data.pw_code, 
    hp_code=data.hp_code, 
    hubbard_structure=hubbard_structure,
    protocol="fast",
    overrides={
        "clean_workdir": False,
        "tolerance_onsite": 0.5,
        "tolerance_intersite": 0.1,
        "relax":{
            "base":{
                "kpoints_distance":100.0,
                "pw":{
                    "parameters":{
                        "SYSTEM":{
                            "ecutwfc": 60.0, # to speed up the tutorial
                            "ecutrho": 60.0 * 8,
                        },
                    },
                },
            }
        }, # to speed up the tutorial
        "scf":{
            "kpoints_distance":100.0, 
            "pw":{
                "parameters":{
                    "SYSTEM":{
                        "ecutwfc": 30.0, # to speed up the tutorial
                        "ecutrho": 30.0 * 8,
                    },
                },
            },
        }, 
        "hubbard":{"qpoints_distance":100.0, "parallelize_atoms":False, "parallelize_qpoints":False}}, # to speed up the tutorial
)

results, node = run_get_node(builder)

Let's inspect the status of the work chain to see the full self-consistency on screen!

In [ ]:
%verdi process status {node.pk}

And of course, here you have the final __relaxed__ structure with __fully self-consistent ab-initio Hubbard parameters__! :tada:

In [ ]:
from aiida_quantumespresso.utils.hubbard import HubbardUtils
print(HubbardUtils(results['hubbard_structure']).get_hubbard_card())

## Final considerations

We managed to compute the Hubbard parameters self-consistently with a series of relaxations, scfs, and hp calculations, ___all fully automated___! :tada:


:::{admonition} Learn more and in details
:class: hint

To learn the full sets of inputs, to use proficiently the `get_builder_from_protocol` and more, have a look at the following sections:
- [Specific how tos](howto/workflows/hubbard.md)
- [General information of the implemented workchain](topics/workflows/hubbard.md)
:::